# Advanced Data Processing and Interactive Visualization

## Working with Multiple Datasets

### Concatenating
Sometimes, we just want to add rows or columns to a table. This is very easy to do!

In [ ]:
import pandas as pd

In [ ]:
# For this example, let's create three small DataFrames
df1 = pd.DataFrame({"FirstName":["Tim", "Melanie", "Markus"], "LastName":["Meier", "Schmidt", "Müller"]})
df2 = pd.DataFrame({"FirstName":["Anna", "Aylin", "Luca"], "LastName":["Wagner", "Yildirim", "Rossi"]})
df3 = pd.DataFrame({"ZipCode":["20095", "70173", "60594"], "Street":["Hohe Straße", "Königsstraße", "Schweizer Straße"], "HouseNumber":["45", "28", "76"]})

In [ ]:
df1

In [ ]:
df2

In [ ]:
df3

We can create a longer DataFrame by concatenating the first two DataFrames vertically (stacking them on top of each other).

In [ ]:
pd.concat([df1, df2], axis=0)

We can also create a wider DataFrame by concatenating them horizontally (placing them side-by-side).

In [ ]:
pd.concat([df1, df3], axis=1)

Now, let's look at a more common and powerful case.

### Joining DataFrames

So far, we've had all our information in a single dataset. But often, data is split across several related tables that we need to work with together. For example, one table might hold product information, a second might have customer information, and a third could track transactions (which links customers to the products they bought).

To avoid duplicating information and to keep the transaction table from getting unnecessarily large, it might only have three essential columns: `CustomerID`, `ProductID`, and `Timestamp`.

If you want to know which warehouse a product is in and where it needs to be shipped, you need what's called a `join`.

A join lets you combine tables. For our example, you could `join` the customer's address from the customer table and the warehouse location from the product table onto the transaction table. This works because the transaction table contains a **unique identifier** for each customer and product.

In this part, we will use data from the Football Champions League ([Source](https://www.kaggle.com/datasets/pabloramoswilkins/ucl-2025-players-data/data)). As you'll see, the information is spread across several different tables instead of being in one large file.

Suppose we want to know which player plays for which team. To find this out, we need to **join** the `players_data` with the `teams_data`.

In [ ]:
import pandas as pd

# As before, let's load the data into separate variables
df_players = pd.read_csv("players_data.csv")
df_teams = pd.read_csv("teams_data.csv")

We need to find an identifier that exists in both tables. In this case, we see `id_team` in the players table and `team_id` in the teams table.

In [ ]:
df_players.head(3)

In [ ]:
df_teams.head(3)

Now we can perform the **join**. We'll use the `pd.merge()` function for this.

In [ ]:
# The order of the DataFrames matters. The first one is the "left" table, and the second is the "right".
pd.merge(df_players, df_teams, how="left", left_on="id_team", right_on="team_id")

Wow, that table has become quite large again. We really only wanted to know which player is on which team. We can select only the columns we want to keep during the join:

In [ ]:
df_player_team = pd.merge(df_players[["id_player", "id_team", "player_name"]], df_teams[["team_id", "team"]], how="left", left_on="id_team", right_on="team_id")
df_player_team.head()

That looks much cleaner, but now we have the team ID in our table twice. We can remove extra columns after the fact.

In [ ]:
df_player_team = df_player_team.drop("team_id", axis=1)
df_player_team.head()

Now we can easily answer which players are on a team's roster.

In [ ]:
df_player_team[df_player_team.team == "FC Bayern München"]

<font color='green'>**Your turn: Using similar techniques and the `goals_data.csv` dataset, find out which player scored the most goals.**</font>

In [ ]:
# Your code here

<font color='green'>**Bonus Task: Which team scored the most headers (German: "Kopfballtore")?**</font>

In [ ]:
# Your code here

<font color='green'>**Advanced Bonus Task: The defenders of which team committed the most fouls?**</font>

In [ ]:
# Your code here

## Playground
<font color='green'>**Feel free to do more analysis here if you like 😊**</font>

In [ ]:
# Your code here

# Interactive Visualization with `altair`
In the past, we learned to use `pandas.plot()`, `matplotlib.pyplot`, and `seaborn` to create static visualizations. We've already used some of those methods in this course.

However, you can often gain more insights from interactive visualizations. For this, Python offers numerous libraries.

In this chapter, we'll focus on `altair`, a Python package that is easy to read and write, yet offers a wide range of possibilities. You can learn more about it [here](https://altair-viz.github.io/) and find many cool examples in the [Altair Example Gallery](https://altair-viz.github.io/gallery/index.html).

This next command shows you which version of `Altair` you are using on your system. This notebook is prepared for **Altair version 5**. The syntax for interactive elements changed significantly from version 4 to 5, so be sure you're looking at the right documentation if you explore on your own!


In [ ]:
import altair as alt
alt.__version__

## The Basics

In [ ]:
import pandas as pd
df_players = pd.read_csv("players_data.csv")

Since we'll be using these columns frequently, it makes sense to rename them to something simpler. We can use `rename()` for this and overwrite the original DataFrame.

In [ ]:
df_players = df_players.rename(columns={"weight(kg)": "weight", "height(cm)":"height"})

When we want to create a chart with Altair, we always start by specifying the data.

A small note: Altair can only display a maximum of 5000 data points by default. If our dataset is larger than that, we can take a random sample by using `df.sample(n=5000, random_state=123)`.

In [ ]:
# This code doesn't work yet. We're not finished with our visualization.
# Don't try fix it - just move on to the next cell.
alt.Chart(df_players)

We still need to choose a visualization type. In `altair`, these are called **[marks](https://altair-viz.github.io/user_guide/marks/index.html)**. We can choose from common types like `bar`, `line`, and `circle`.

In [ ]:
# We still haven't told Altair which columns to plot.
# So, it just displays a single point.
alt.Chart(df_players).mark_circle()

So, let's select the columns as well. We do this with the `.encode()` method. We also specify the data type as `"quantitative"` for our numerical data. Alternatively, we could use `"nominal"` for categorical data or `"ordinal"` for ordered data.

<font color='green'>**Try changing the data type of a variable and see what happens.**</font>

You can read more about this under "Encoding Data Types" on [this page](https://altair-viz.github.io/user_guide/encodings/index.html).

In [ ]:
# Since our first little visualization is complete, we can save it in a variable and call it again.
chart = alt.Chart(df_players).mark_circle().encode(
    alt.X("height", type="quantitative"),
    alt.Y("weight", type="quantitative")
)

chart

We can make our chart a bit more interesting by using color to distinguish a player's position.

In [ ]:
# We can add more specifications in another encode call.
chart.encode(alt.Color("field_position", type="nominal", title="Field Position"))

<font color='green'>**Try to build a similar visualization with two other numerical columns.**</font>

In [ ]:
# Your code here

## Simple Interactivity
So far, the visualization has been static. But we can add some basic interactive elements very easily.

In [ ]:
# Recreating the previous chart, first.
chart = alt.Chart(df_players).mark_circle().encode(
    alt.X("height", type="quantitative"),
    alt.Y("weight", type="quantitative"),
    alt.Color("field_position", type="nominal", title="Field Position")
)

chart

If we call the chart with the `.interactive()` suffix, we can zoom in and pan around the image with the mouse.

<font color='green'>**Try it out.**</font>

In [ ]:
chart.interactive()

This allows us to better distinguish the positions. However, we still don't know which point represents which player. For that, we can use a tooltip.

We can either specify it when creating the graph or, again, add it with an additional encoding to the existing chart.

In [ ]:
# Now you can hover over the points with your mouse and see the player's name.
chart.encode(tooltip=["player_name"])

However, we've now lost the ability to zoom. To get that back, we need to call the chart in interactive mode again.

<font color='green'>**Combine the tooltip with `.interactive()` to use both effects at the same time.**</font>

In [ ]:
# Your code here

You might have noticed that we provide a list to the tooltip. This means we can easily display multiple pieces of information on mouseover.

<font color='green'>**Think about what additional column you would like to display as a tooltip.**</font>

Advanced challenge: Display the team a player belongs to.

In [ ]:
# Your code here

## Aggregation
So far, we have been displaying raw data, but often we want to show aggregated data, like the mean or standard deviation.

Note: This could all be done in `pandas` beforehand, but it can also be done directly in `altair`. You can find more details [here on transformations](https://altair-viz.github.io/user_guide/transform/aggregate.html) and [here on encoding aggregates](https://altair-viz.github.io/user_guide/encodings/index.html#encoding-aggregates).

Suppose we don't want to show every single player, but the average values per position. For this, we use an aggregation.

In [ ]:
chart = alt.Chart(df_players).mark_bar().encode(
    alt.X("height", type="quantitative", aggregate="average"), # We aggregate our x-variable with the mean
    alt.Y("field_position", type="nominal")
)

chart

<font color='green'>**Try to do the same with weight.**</font>

<font color='green'>**Optional: Select the tallest or shortest player per position. To do this, use `aggregate="min"` or `aggregate="max"`.**</font>

In [ ]:
# Your code here

If we want to use the same type of display to show the average age per nationality, it could get a bit cluttered because we have so many groups. Here, a *filter* can be useful. You can find more about this [here](https://altair-viz.github.io/user_guide/transform/filter.html).

In [ ]:
# Let's find the top 5 most common nationalities in the dataset
df_players.nationality.value_counts()[:5]

In [ ]:
# We select the desired columns
# Then we add a transform_filter
chart = alt.Chart(df_players).mark_bar().encode(
    alt.X("age", type="quantitative", aggregate="average"),
    alt.Y("nationality", type="nominal")
).transform_filter(
    alt.FieldOneOfPredicate(field='nationality', oneOf=["France", "Spain", "Germany", "Italy", "Netherlands"])
)

chart

If we want to sort the chart now, we can also specify that easily.

In [ ]:
chart = alt.Chart(df_players).mark_bar().encode(
    alt.X("age", type="quantitative", aggregate="average"),
    alt.Y("nationality", type="nominal", sort="-x") # We specify here what to sort the Y-axis by. In this case, x, descending.
).transform_filter(
    alt.FieldOneOfPredicate(field='nationality', oneOf=["France", "Spain", "Germany", "Italy", "Netherlands"])
)

chart

<font color='green'>**Advanced Bonus Challenge: We want to know where the stars of tomorrow come from. Create a chart showing the number of players aged 18-21 per nationality in our top 5.**</font>

Hint: Use the [`distinct` aggregation](https://altair-viz.github.io/user_guide/encodings/index.html#encoding-aggregates) and the [`FieldRangePredicate()` filter](https://altair-viz.github.io/user_guide/transform/filter.html).

In [ ]:
# Your code here

## Advanced Interactive Visualization

**Important Note:** Especially with interactive elements, Altair version 4 differs greatly from Altair version 5. So, pay attention to the correct version when reading the documentation.

### Interactive Selections
`Altair` allows us to define sliders, dropdowns, and similar elements to dynamically adjust our visualizations. You can find examples of these [here](https://altair-viz.github.io/gallery/index.html#gallery-category-interactive-charts) and read the technical details [here](https://altair-viz.github.io/user_guide/interactions.html).

Let's go back to the chart from above:

In [ ]:
chart = alt.Chart(df_players).mark_bar().encode(
    x = alt.X("height", type="quantitative", aggregate="average"),
    y = alt.Y("field_position", type="nominal", sort="-x"),
    color = alt.value("lightgray") # set a default color
)

chart

We can dynamically change the color based on the mouse position:

In [ ]:
# Create a selection that updates on mouseover
point_selector = alt.selection_point(on='mouseover', nearest=True)

chart.encode(
  # If a point is selected, color it salmon, otherwise lightgray
  color = alt.condition(point_selector, alt.value("salmon"), alt.value('lightgray'))
).add_params(point_selector) # Add the selector to the chart

What happened in the code above? We created a `selection_point` and made the color dependent on a condition. The `point_selector` is either `True` or `False` for a specific bar. Similar to an IF function in Excel, if `True`, the first value is used (i.e., "salmon"), and if `False`, the value "lightgray" is used.

<font color='green'>**Try deleting the arguments `on` and `nearest` to see what happens. Optionally, use different colors.**</font>

In [ ]:
# Your code here

However, using **bindings** like dropdowns, checkboxes, or sliders is often more practical. You can learn more about these in the Altair documentation on [bindings](https://altair-viz.github.io/user_guide/interactions.html#input-element-binding).

Let's take the scatter plot from before. I've adjusted the circle `size` and the axes so we can see the points better.

In [ ]:
chart = alt.Chart(df_players).mark_circle(size=100).encode(
    x = alt.X("height", type="quantitative", scale=alt.Scale(domain=[160, 200])),
    y = alt.Y("weight", type="quantitative", scale=alt.Scale(domain=[40, 100])),
    color = alt.Color("field_position", type="nominal", title="Position")
)

chart

Now, let's create the chart with a dropdown menu for the player's position.

In [ ]:
# Create a dropdown with the possible positions
input_dropdown = alt.binding_select(options=['Forward', 'Midfielder', 'Defender', 'Goalkeeper', None], name='Position ')

# Create a selection object that checks the value of the dropdown
selection = alt.selection_point(fields=['field_position'], bind=input_dropdown)

# Define the color based on our selection object
color = alt.condition(selection,
                    alt.Color('field_position:N', legend=None),
                    alt.value('lightgray'))


scatter_plot = alt.Chart(df_players).mark_circle(size=100).encode(
    x = alt.X("height", type="quantitative", scale=alt.Scale(domain=[160, 200])),
    y = alt.Y("weight", type="quantitative", scale=alt.Scale(domain=[40, 100])),
    color = color # Here we use our predefined color
).add_params(
    selection
)

scatter_plot

<font color='green'>**Advanced Task: Create a similar visualization with 5 nationalities of your choice. Only the nationality selected in the dropdown should be highlighted in color.**</font>

In [ ]:
# Your code here

### Combining Charts
Altair lets us combine charts in any way we like. This allows us to build complex visualizations that show different facets of the dataset at the same time.

In [ ]:
# We create the same bar chart, but select the color using the previously created dropdown
bar_chart_height = alt.Chart(df_players).mark_bar().encode(
    x = alt.X("height", type="quantitative", aggregate="average"),
    y = alt.Y("field_position", type="nominal", sort="-x"),
    color = color # We use the same color condition
).add_params(selection) # And the same selection parameter


# We combine the two charts vertically
alt.vconcat(scatter_plot, bar_chart_height)

# Playground
<font color='green'>**Continue experimenting with (interactive) visualizations if you like 😊**</font>

In [ ]:
# Your code here